# **Bayesian Radioactive Decay Inference (2D Grid)**

This notebook visualize the simplified radioactive decay problem in Homework 3.

## Enable Autoreload

Enabling autoreload make debugging much easier.
However, there is a bug in `ipython 9` that makes autoreload fail.
Hence, we will use a exception to try enabling autoreload.

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
except ModuleNotFoundError as e:
    print(e)
    print("Probably bug from ipython.")
    print("Relaunch kernels after editing codes, or downgrade your ipython/jupyterlab.")

## Load Python Package

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
! pip install -e .. # Install the homework package if needed

In [ ]:
from phys305_hw3 import *

## Set up the Test Case

In [ ]:
# Fix random seed for reproducibility

np.random.seed(42)

In [ ]:
# Parameter

n0    = 100
lmbda = 0.01

In [ ]:
# Sampling Time

ts = np.linspace(0, 100, 11)
dt = 10

## Test Codes in "a1.py"

In [ ]:
groundtruth = n0, lmbda

Cts     = count (ts, dt, groundtruth)
Cts_obs = sample(ts, dt, groundtruth)

print("Times           :", ts)
print("Expected counts :", Cts)
print("Observed counts :", Cts_obs)

In [ ]:
plt.plot   (ts, Cts,            'o-', color='C0', label="Expected Counts")
plt.scatter(ts, Cts_obs, marker='x',  color='C1', label="Observed Counts")

plt.xlabel("Time")
plt.ylabel("Counts per Interval")
plt.title("Synthetic Data")
plt.legend()

## Test Codes in "a2.py"

In [ ]:
# Save prior of n0 on a grid and plot it

n0s, ps_n0 = prior0_n0(0, 300, 31)
plt.plot(n0s, ps_n0, 'o-')

In [ ]:
# Save prior of lmbda on a grid and plot it

lmbdas, ps_lmbda = prior0_lmbda(1e-4, 1, 41)
plt.semilogx(lmbdas, ps_lmbda, 'o-')

In [ ]:
# Look at the shape and dimensions of these grids

print(n0s.shape)
print(lmbdas.shape)

In [ ]:
# When we need to create 2D grid, it is useful to learn the `np.meshgrid()` function.
# Use it and check the array dimensions.

params = np.meshgrid(n0s, lmbdas)

n0ss, lmbdass = params
print(n0ss.shape)
print(lmbdass.shape)

In [ ]:
# It is also useful to learn numpy broadcasting:
#   https://numpy.org/doc/stable/user/basics.broadcasting.html
# and the np.newaxis syntax.
# The follow example ensures `prior` has same dimensions as `n0ss` and `lmbdass`.

prior = ps_n0[np.newaxis,:] * ps_lmbda[:,np.newaxis]
print(prior.shape)

In [ ]:
plt.pcolormesh(n0s, lmbdas, prior)
plt.yscale('log')
plt.xlabel(r'$n_0$')
plt.ylabel(r'$\lambda$')

## Test Codes in "a3.py"

In [ ]:
obs = Cts, ts, dt

like = likelihood(obs, params)

In [ ]:
plt.pcolormesh(n0s, lmbdas, like)
plt.yscale('log')
plt.xlabel(r'$n_0$')
plt.ylabel(r'$\lambda$')